In [ ]:
!pip install python-pptx sentence-transformers chromadb

In [1]:
from pptx import Presentation
import os
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
from openai import OpenAI
from typing import List, Dict, Tuple

In [2]:
class CourseQASystem:
    def __init__(self):
        # 初始化组件
        self.vector_db = self._init_vector_db()
        self.embed_model = SentenceTransformer("BAAI/bge-small-zh-v1.5")
        self.llm_client = OpenAI(
            api_key=os.environ.get("DEEPSEEK_API_KEY"),
            base_url="https://api.deepseek.com/v1"
        )

        # 对话历史存储 {session_id: [(query, response), ...]}
        self.dialog_histories = {}

    def _init_vector_db(self):
        """初始化向量数据库"""
        client = chromadb.PersistentClient(path="./vector_db")
        try:
            client.delete_collection("course_qa")
        except:
            pass
        return client.create_collection(name="course_qa")

    def _retrieve_with_history(self, query: str, session_id: str, n_results: int = 3) -> str:
        """结合对话历史的增强检索"""
        # 获取最近3轮对话历史
        history = self.dialog_histories.get(session_id, [])[-3:]

        # 构建上下文增强的查询
        context_queries = [query]
        for q, a in history:
            context_queries.extend([q, a])

        # 生成混合查询向量
        mixed_query = " ".join(context_queries)
        query_embedding = self.embed_model.encode([mixed_query], normalize_embeddings=True)[0].tolist()

        # 执行检索
        results = self.vector_db.query(
            query_embeddings=[query_embedding],
            n_results=n_results + len(history),  # 动态调整结果数量
            include=["documents"]
        )
        return "\n\n".join(results["documents"][0])

    def generate_response(self, query: str, session_id: str = "default") -> str:
        """处理带历史上下文的问答"""
        # 检索相关知识
        context = self._retrieve_with_history(query, session_id)

        # 获取对话历史（最近3轮）
        history = self.dialog_histories.get(session_id, [])[-3:]

        # 构建提示模板
        prompt = f"""你是一个智能课程助教，请严格根据以下课程内容和对话历史回答问题。
        
        课程内容片段：
        {context}
        
        对话历史：
        {self._format_history(history)}
        
        当前问题：{query}
        
        请用简洁的中文回答，如果无法确定答案请说明。避免使用Markdown格式。"""

        # 调用大模型
        response = self.llm_client.chat.completions.create(
            model="deepseek-chat",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=512
        )

        answer = response.choices[0].message.content

        # 更新对话历史
        if session_id not in self.dialog_histories:
            self.dialog_histories[session_id] = []
        self.dialog_histories[session_id].append((query, answer))

        return answer

    def _format_history(self, history: List[Tuple[str, str]]) -> str:
        """格式化对话历史"""
        return "\n".join([f"Q: {q}\nA: {a}" for q, a in history])

    def clear_history(self, session_id: str = "default"):
        """清空指定会话历史"""
        if session_id in self.dialog_histories:
            del self.dialog_histories[session_id]

In [3]:
def initialize_system():
    # 1. PPT处理流程
    os.makedirs("ppt_files", exist_ok=True)

    def process_ppt_files():
        """完整的PPT处理流水线"""
        # 提取原始文本
        all_text = []
        for filename in os.listdir("ppt_files"):
            if filename.endswith(".pptx"):
                ppt_path = os.path.join("ppt_files", filename)
                try:
                    prs = Presentation(ppt_path)
                    file_text = []
                    for slide in prs.slides:
                        for shape in slide.shapes:
                            if shape.has_text_frame:
                                for paragraph in shape.text_frame.paragraphs:
                                    file_text.append(paragraph.text)
                    merged_text = "\n".join(file_text)
                    all_text.append(f"=== {filename} ===\n{merged_text}\n\n")
                except Exception as e:
                    print(f"处理文件 {filename} 时出错: {e}")

        # 清洗文本
        def clean_text(text):
            text = re.sub(r'\n+', '\n', text)
            text = re.sub(r' +', ' ', text)
            text = re.sub(r'\d+/\d+', '', text)
            text = re.sub(r'[^\w\u4e00-\u9fff\s.,;:!?()%+-]', '', text)
            return text.strip()

        raw_text = "\n".join(all_text)
        clean_data = clean_text(raw_text)

        # 文本分块
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=30,
            separators=["\n\n", "\n", "。", "；", "，"]
        )
        chunks = splitter.split_text(clean_data)

        # 生成并存储向量
        embed_model = SentenceTransformer("BAAI/bge-small-zh-v1.5")
        embeddings = embed_model.encode(chunks, normalize_embeddings=True)

        # 初始化向量数据库
        client = chromadb.PersistentClient(path="./vector_db")
        try:
            client.delete_collection("course_qa")
        except:
            pass
        collection = client.create_collection(name="course_qa")

        # 批量添加数据
        collection.add(
            documents=chunks,
            embeddings=[vec.tolist() for vec in embeddings],
            ids=[f"chunk_{i}" for i in range(len(chunks))]
        )
        print(f"成功存储 {len(chunks)} 个文本块到向量数据库")
        return True

    # 执行PPT处理
    if os.path.exists("ppt_files") and len(os.listdir("ppt_files")) > 0:
        print("开始处理PPT文件...")
        process_ppt_files()
    else:
        print("警告：ppt_files目录为空或不存在，请先添加PPT文件")

    # 2. 初始化问答系统
    qa_system = CourseQASystem()
    return qa_system


In [4]:
if __name__ == "__main__":
    # 初始化系统
    qa = initialize_system()
    print("""\n=== 智能课程问答系统 ===
支持功能：
1. 多轮对话（自动记忆最近3轮历史）
2. 检索增强生成（RAG）
3. 输入'clear'清空对话历史
4. 输入'exit'退出系统\n""")

    session_id = "default"
    while True:
        try:
            # 获取用户输入
            query = input("\n请输入问题 (输入exit退出): ").strip()

            # 退出判断
            if query.lower() in ["exit", "quit"]:
                print("\n=== 退出系统 ===")
                break

            # 清空历史命令
            if query.lower() == "clear":
                qa.clear_history(session_id)
                print("[系统] 已清空对话历史")
                continue

            # 空输入处理
            if not query:
                print("[系统] 请输入有效问题")
                continue

            # 生成回答
            print("\n[系统] 思考中...", end="", flush=True)
            response = qa.generate_response(query, session_id)
            print("\r[系统]", response)  # \r用于覆盖"思考中"提示

        except KeyboardInterrupt:
            print("\n\n=== 使用Ctrl+C退出请使用exit命令 ===")
        except Exception as e:
            print(f"\n[系统错误] 发生异常: {str(e)}")

    # 展示最终对话历史
    if session_id in qa.dialog_histories:
        print("\n=== 完整对话历史 ===")
        for i, (q, a) in enumerate(qa.dialog_histories[session_id], 1):
            print(f"\n第{i}轮对话：")
            print(f"问：{q}")
            print(f"答：{a}")

开始处理PPT文件...
成功存储 121 个文本块到向量数据库

=== 智能课程问答系统 ===
支持功能：
1. 多轮对话（自动记忆最近3轮历史）
2. 检索增强生成（RAG）
3. 输入'clear'清空对话历史
4. 输入'exit'退出系统



=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===


=== 使用Ctrl+C退出请使用exit命令 ===

=== 退出系统 ===
